In [2]:
!pip install outlines -q
!pip install -q transformers 
!pip install -q openai 
!pip install -q datasets
!pip install -q accelerate
!pip install trieregex


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached trieregex-1.0.0-py3-none-any.whl

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from enum import Enum
from pydantic import BaseModel, constr

import outlines.models as models
import outlines.text.generate as generate

import torch


model = models.transformers("gpt2")#, device="cuda")
# model = models.transformers("Open-Orca/Mistral-7B-OpenOrca", device="cuda")


# json

In [4]:

class Weapon(str, Enum):
    sword = "sword"
    axe = "axe"
    mace = "mace"
    spear = "spear"
    bow = "bow"
    crossbow = "crossbow"


class Armor(str, Enum):
    leather = "leather"
    chainmail = "chainmail"
    plate = "plate"


class Character(BaseModel):
    name: constr(max_length=10)
    age: int
    armor: Armor
    weapon: Weapon
    strength: int

    
# Construct guided sequence generator
generator = generate.json(model, Character, max_tokens=100)

# Draw a sample
rng = torch.Generator(device="cuda")
rng.manual_seed(789001)

sequence = generator("Give me a character description", rng=rng)
print(sequence)
# {
#   "name": "clerame",
#   "age": 7,
#   "armor": "plate",
#   "weapon": "mace",
#   "strength": 4171
# }

sequence = generator("Give me an interesting character description", rng=rng)
print(sequence)
# {
#   "name": "piggyback",
#   "age": 23,
#   "armor": "chainmail",
#   "weapon": "sword",
#   "strength": 0
# }

parsed = Character.model_validate_json(sequence)
print(parsed)
# name='piggyback' age=23 armor=<Armor.chainmail: 'chainmail'> weapon=<Weapon.sword: 'sword'> strength=0

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(Record(f0[type=int64;offset=0],f1[type=int64;offset=8];16;False), Literal[int](0))
 
There are 22 candidate implementations:
[1m      - Of which 22 did not match due to:
      Overload of function 'getitem': File: <numerous>: Line N/A.
        With argument(s): '(Record(f0[type=int64;offset=0],f1[type=int64;offset=8];16;False), int64)':[0m
[1m       No match.[0m
[0m
[0m[1mDuring: typing of intrinsic-call at /opt/conda/lib/python3.8/site-packages/outlines/text/fsm.py (109)[0m
[0m[1mDuring: typing of static-get-item at /opt/conda/lib/python3.8/site-packages/outlines/text/fsm.py (109)[0m
[1m
File "../../../../../opt/conda/lib/python3.8/site-packages/outlines/text/fsm.py", line 109:[0m
[1mdef create_fsm_info(
    <source elided>
        trans_key_to_states.setdefault(
[1m            trans_k

# regex

In [137]:
prompt = "Is 1+1=2? "
unguided = generate.continuation(model, max_tokens=30)(prompt)
guided = generate.regex(model, r"\s*([Yy]es|[Nn]o|[Nn]ever|[Aa]lways)", max_tokens=30)(
    prompt
)

print(unguided)
# Is 1+1=2?
#
# This is probably the most perplexing question.
# As I said in one of my articles describing how
# I call 2 and 1, there isn't

print(guided)
# Is 1+1=2? Always

icalnt) rape: ("fucking being raped",...1+1=2?), "acting the part" (C45), that is,


always


## creating regex pattern

In [142]:
import random
import string
import re
from trieregex import TrieRegEx as TRE

# Function to generate a random word
def generate_random_word(length):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

# Generate 20,000 random words
random_words = [generate_random_word(random.randint(3, 10)) for _ in range(3)]

# You can now use the 'random_words' list as needed.
# For example, to print the first 10 words:
print(random_words[:10])
print(len(random_words))

# Initialize class instance
tre = TRE()

# Add word(s)
tre = TRE(*random_words)

print(len(tre.regex()))


['xfpu', 'tdr', 'azflyun']
3
20


In [143]:
def generate_regex_pattern(word_list):
    # Start with an optional whitespace character
    # regex_pattern = r'\s*('
    regex_pattern = r'('

    # Create a non-capturing group for each word, allowing any case for the first letter
    transformed_words = []
    for word in word_list:
        if word[0].isupper():  # If the first letter is uppercase
            transformed_word = f"[{word[0].lower()}{word[0]}]{word[1:]}"
        else:  # If the first letter is lowercase
            transformed_word = f"[{word[0]}{word[0].upper()}]{word[1:]}"
        transformed_words.append(transformed_word)

    # Join the transformed words with the pipe symbol (which stands for "or" in regex)
    regex_pattern += '|'.join(transformed_words)
    
    # Close the group
    regex_pattern += ')'
    
    return regex_pattern

In [144]:


# Given list of words
# words = ["yes", "no", "Never", "always"]
words = random_words
len(words)
# Generate the regex pattern
regex_pattern = generate_regex_pattern(words)
len(regex_pattern)

27

In [145]:
prompt = "Is 1+1=2? "
unguided = generate.continuation(model, max_tokens=1)(prompt)

print(unguided)
# Is 1+1=2?
#
# This is probably the most perplexing question.
# As I said in one of my articles describing how
# I call 2 and 1, there isn't


!!!


In [146]:
guided = generate.regex(model, regex_pattern, max_tokens=4)(
    prompt
)
# take the statistics . for such questions it may be useful to take the top probability word 
print(guided)
# Is 1+1=2? Always

xfpu


In [148]:
# random_words
guided = generate.choice(model, choices=words, max_tokens=4)(
    prompt
)
# take the statistics . for such questions it may be useful to take the top probability word 
print(guided)
# Is 1+1=2? Always

azfly


In [12]:
special_tokens = tokenizer.all_special_tokens
special_tokens

['<s>', '</s>', '<unk>']

In [14]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer



# Load the model and tokenizer
# model = GPT2LMHeadModel.from_pretrained("gpt2")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# mistralai/Mistral-7B-Instruct-v0.1

# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", torch_dtype="auto")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", torch_dtype="auto")


# Encode the input text
input_text = "The current weather is"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Get the model's output (logits)
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0]

# Apply softmax to convert logits to probabilities
softmax = torch.nn.Softmax(dim=-1)
probabilities = softmax(predictions)
special_tokens = tokenizer.all_special_tokens

# Iterate through each word in the input and check the prediction for the next word
for i, input_id in enumerate(input_ids[0]):
    if tokenizer.decode([input_id]) in special_tokens:
        # Skip special tokens
        continue

    if i < len(input_ids[0]) - 1:
        # Get the index of the actual next word
        actual_next_word_id = input_ids[0][i+1].item()
        # Get the probability distribution for the next word
        next_word_probabilities = probabilities[0, i]
        # Get the predicted token ID with the highest probability for the next word
        predicted_next_word_id = next_word_probabilities.argmax().item()
        
        # Decode the predicted and actual next words
        predicted_token = tokenizer.decode([predicted_next_word_id])
        actual_next_word = tokenizer.decode([actual_next_word_id])
        
        print(f"Input token: '{tokenizer.decode([input_id])}'")
        print(f"Predicted next token: '{predicted_token}'")
        print(f"Actual next token: '{actual_next_word}'")
        print()

# Handle the last token prediction
last_word_probabilities = probabilities[0, -1]
predicted_next_word_id_for_last = last_word_probabilities.argmax().item()
predicted_token_for_last = tokenizer.decode([predicted_next_word_id_for_last])

print(f"Last input token: '{tokenizer.decode([input_ids[0][-1].item()])}'")
print(f"Predicted next token: '{predicted_token_for_last}'")


Input token: 'The'
Predicted next token: ''
Actual next token: 'current'

Input token: 'current'
Predicted next token: 'state'
Actual next token: 'weather'

Input token: 'weather'
Predicted next token: 'forecast'
Actual next token: 'is'

Last input token: 'is'
Predicted next token: 'a'


In [6]:
# Get the logits for the last token in the input
last_token_logits = predictions[:, -1, :]

# Define candidate words
candidate_words = ["sunny", "rainy", "cloudy", "windy", 'not']

# Retrieve the indices of the candidate words
candidate_indices = [tokenizer.encode(word)[0] for word in candidate_words]

# Extract the logits for the candidate words
candidate_logits = last_token_logits[:, candidate_indices]

# Find the most likely candidate word
most_likely_word_index = candidate_logits.argmax(-1).item()
predicted_word = candidate_words[most_likely_word_index]

print(predicted_word)

not


In [7]:
most_likely_word_index

4

In [8]:
candidate_logits.shape

torch.Size([1, 5])

In [9]:
!pip install  -q pygtrie


In [11]:
import pygtrie

def build_trie_from_document(document):
    words = document.split()  # Split the document into words
    trie = pygtrie.CharTrie()
    
    for i in range(len(words) - 1):
        prefix = words[i]
        next_word = words[i + 1]
        
        if prefix in trie:
            trie[prefix].add(next_word)
        else:
            trie[prefix] = {next_word}
    
    return trie


# Usage
document = "this is a test document with the test document having some test cases"
trie = build_trie_from_document(document)

word = "test"
next_words = trie.get(word)
if next_words is not None:
    print(f"Next words for '{word}': {next_words}")
else:
    print(f"No next words for '{word}' found.")


Next words for 'test': {'document', 'cases'}


In [16]:
trie['this']

{'is'}

In [17]:
trie['is']

{'a'}

In [18]:
trie['a']

{'test'}

# mistral gpu

In [2]:
!pip install -q ipywidgets widgetsnbextension pandas-profiling




[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
torch.set_default_device('cuda')

In [5]:


model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1",
                                             torch_dtype="auto")
# mistralai/Mistral-7B-Instruct-v0.1
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1",
                                          torch_dtype="auto")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"
text = 'hello'
encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

device = 'cuda'
model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=10, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


hello-world/master

## Getting started and


In [ ]:
print('htis')